# Lectura 2: Recuperación de la información

In [ ]:
import re
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from  sklearn import preprocessing
from scipy.spatial import distance

En la siguiente clase se encuentra el código para realizar el conteo de bolsas de palabras $X_{d}$ dados $D$ documentos

In [ ]:
class Lectura2:
    
    def __init__(self,d,Y):
        #Vector de documentos D
        self.X = d
        #Vector de conteo
        self.Xd = ' '.join([palabra for palabra in self.X])
        #Consulta
        self.Y = Y
        #Vector de consulta
        self.XY = list()
        [self.XY.append(x) for x in self.X]
        self.XY.append(Y)
        self.Xdw = list()
        #matriz de terminos del documento
        self.mtc = list()
        self.indices = ['X'+str(i) for i in range(1,len(self.X)+1)]
        self.indices.append('Y')
        
    def procesa_palabras(self):
        [self.Xdw.append(palabra) for palabra in self.Xd.split(' ') if palabra not in self.Xdw]
        self.Xdw = sorted(self.Xdw)
        
    def matriz_terminos(self):
        matriz_terminos = list()
        l = len(self.XY)
        for t in range(0,l):
            matriz_terminos.append([0]*len(self.Xdw))
    
        for k,documentos in enumerate(self.XY):
            for v in documentos.split(' '):
                if(v in self.Xdw):
                    matriz_terminos[k][self.Xdw.index(v)]+=1
        self.mtc = matriz_terminos
        
        return pd.DataFrame(matriz_terminos,columns=self.Xdw, index=self.indices)
    
    def get_matriz_terminos(self):
        return self.mtc
    
    def distancia_euclidiana(self,matriz,columna):
        distancia = list()
        l = len(matriz)
        for i in range(0,l):
            distancia.append(distance.euclidean(tuple(matriz[i]),
                                     tuple(matriz[l-1])))
        return pd.DataFrame(distancia,columns=[columna], index=self.indices)
    
    def distancia_manhattan(self,matriz,columna):
        distancia = list()
        l = len(matriz)
        for i in range(0,l):
            distancia.append(distance.cityblock(tuple(matriz[i]),
                                     tuple(matriz[l-1])))
        return pd.DataFrame(distancia,columns=[columna], index=self.indices)
    
    def normalizacion_documento(self,matriz):
        longitudes = list()
        l = len(matriz)
        for i in range(0,l):
            norma = np.divide(matriz[i],sum(matriz[i]))
            longitudes.append(norma)
        #print(longitudes)
        return longitudes
    def normalizacion_l2(self,matriz):
        longitudes = list()
        l = len(matriz)
        for i in range(0,l):
            X_normalizado = preprocessing.normalize(matriz[i], norm='l2')
            longitudes.append(X_normalizado.T)
        return longitudes

**Ejercicio 1** con dos oraciones como $D$ documentos

In [ ]:
d = ['Los alumnos aman la minería de datos',
    'El profesor y los alumnos odian realmente odian la minería de datos']

Y = 'odian minería'
ejercicio = Lectura2(d,Y)
ejercicio.procesa_palabras()
datos = ejercicio.matriz_terminos()
mtc = ejercicio.get_matriz_terminos()
distancia_euclidiana = ejercicio.distancia_euclidiana(mtc,'Distancia Euclidiana')
distancia_manhattan = ejercicio.distancia_manhattan(mtc,'Distancia Manhattan')
print(distancia_euclidiana)
print(distancia_manhattan)

**Ejercicio 2** con n documentos $D$ en un directorio

In [ ]:
directorio = os.path.normpath("documentos")
docs = list()
archivos = [join('documentos', f) for f in listdir('documentos') if isfile(join('documentos', f))]
for archivo in archivos:
    with open(archivo,'rb') as a:
        #print(a.read())
        docs.append(a.read().decode('utf8'))
Y1 = 'inteligencia OSINT información documentos'

In [ ]:
ejercicio2 = Lectura2(docs,Y1)
ejercicio2.procesa_palabras()
datos = ejercicio2.matriz_terminos()
matriz_terminos = ejercicio2.get_matriz_terminos()
distancia = ejercicio2.distancia_euclidiana(matriz_terminos,'Distancia Euclidiana')
distancia_manhattan = ejercicio2.distancia_manhattan(matriz_terminos,'Distancia Manhattan')
norm_documento = ejercicio2.normalizacion_documento(matriz_terminos)
norm_l2 = ejercicio2.normalizacion_l2(matriz_terminos)
distancia_norm = ejercicio2.distancia_euclidiana(norm_documento,'NormaDoc')
distancia_norm_l2 = ejercicio2.distancia_euclidiana(norm_l2,'Distancia Euclidiana/NormaL2')

**Ejercicio 3** Encontrar información de la MISTI en tres diferentes fuentes de internet, guardar el texto citado de cada una en un diferentes archivos *.txt* y realizar por lo menos dos consultas diferentes, obteniendo la distancia Euclidiana y Manhattan sin normalizar $X_{d}$ y dos diferentes con $X_{d}$ normalizado por longitud y por $\ell_{2}$